\# Введение в анализ данных
## НИУ ВШЭ, 2019-2020 учебный год

### Домашнее задание №3

Задание выполнил(а): Колесов Даниил

### Общая информация
Дата выдачи: 

Дедлайн: 

### О задании

В этом домашнем задании вы будете работать с линейной классификацией, попрактикуетесь на реальной задаче классификации текстов.

Для решения этого домашнего задания намного удобнее будет использовать Colab, так как данных много.

### Оценивание и штрафы

За сдачу задания позже срока на итоговую оценку за задание накладывается штраф в размере 1 балл в день, но получить отрицательную оценку нельзя.

__Внимание!__ Домашнее задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов.

### Формат сдачи
Загрузка файлов с решениями происходит в системе Anytask.

### Подготовка данных

In [0]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm

In [76]:
# Датасет можно скачать здесь

!wget https://www.dropbox.com/s/tg55q9mrziroyrs/train_subset.csv

--2020-04-26 18:53:36--  https://www.dropbox.com/s/tg55q9mrziroyrs/train_subset.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/tg55q9mrziroyrs/train_subset.csv [following]
--2020-04-26 18:53:36--  https://www.dropbox.com/s/raw/tg55q9mrziroyrs/train_subset.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5f2e8b86150ae4e7afc97086b6.dl.dropboxusercontent.com/cd/0/inline/A2m-cvXiYcSanXf7RT2vSxUx0DQM60S2MFfSj6i8FsqifyOvZbBSkI8hIC9GVJRvhbEAgwNZt433E2yakIHec_QBpYs1kvA5bczobkpXemeTMsAfnwHjo8Wsc47vPB0CrZY/file# [following]
--2020-04-26 18:53:37--  https://uc5f2e8b86150ae4e7afc97086b6.dl.dropboxusercontent.com/cd/0/inline/A2m-cvXiYcSanXf7RT2vSxUx0DQM60S2MFfSj6i8FsqifyOvZbBSkI8hIC9GVJRvhbEAgwNZt433E2yakIHec_QBpYs1kvA5bcz

### Данные

Мы имеем дело с данными с торговой платформы Avito.
Для каждого товара представлены следующие параметры:
 - title
 - description
 - Category_name
 - Category

Имеется информация об объектах 50 классов.
Задача: по новым объектам (title, description) предсказать Category.
(Очевидно, что параметр Category_name для предсказания классов использовать нельзя)

In [77]:
data = pd.read_csv("train_subset.csv", index_col='id')

data.head()

,title,description,Category_name,Category
id,,,,
382220,Прихожая,В хорошем состоянии. Торг,Мебель и интерьер,20
397529,Кордиант 215/55/16 Летние,Кордиант 215/55/16 Летние/\n /\nАртикул: 1737l...,Запчасти и аксессуары,10
584569,Стол,"Стол, 2 рабочих места . Стол серого цвета, в д...",Мебель и интерьер,20
2513100,Комбинезон,Размер-42/44,"Одежда, обувь, аксессуары",27
1091886,Ветровка,На 2 года,Детская одежда и обувь,29


In [78]:
data.shape

(30000, 4)

In [0]:
X = data[['title', 'description']].to_numpy()
y = data['Category'].to_numpy()

del data

Сразу разделим выборку на train и test.
Никакие данные из test для обучения использовать нельзя!

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [59]:
X_train[:5]

array([['Сапоги 46 размер новые', 'Сапоги 46 размер новые'],
       ['Светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку. В эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iPhone 7 plus 128GB Red красный в наличии',
        '\xa0/\n/\n Данная цена только для подписчиков Instagram: iQmac/\n/\n Новый красный айфон 7 Plus в наличии это элегантный и мощный смартфон, который готов в полной мере раскрыть новые возможности iOS 10. Аппарат с 4-ядерным процессором А10 и 3 ГБ ОЗУ с легкостью решает самые ресурсоемкие задачи, позволяя наслаждаться быстродействием «тяжелых» приложений и игр на 5,5-дюймовом дисплее. Аппарат получил экран, как у iPad Pro, так что картинка теперь соответствует кинематографическому стандарту.'],
       ['Пион Ирис Ромашка рассада',
        'Пион куст 500 р ( более 10 шт)/\nСаженец/ корень 100р/\nРастут у нас более 70 лет/\nРозовые, бордовые и белые/\nНа фото цветы 2018г/\nП. Зубчанинов

### Токенизация (1 балл)


Токенизация -- разбиение текста на мелкие части, которые можно обработать машинными методами.
Можно использовать разные алгоритмы токенизации.
Давайте пока остановимся на простом WordPunctTokenizer.


In [9]:
from nltk.tokenize import WordPunctTokenizer


tokenizer = WordPunctTokenizer()


def preprocess(text: str) -> str:
    return ' '.join(tokenizer.tokenize(text.lower()))

text = 'Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...'
print("before:", text,)
print("after:", preprocess(text))

before: Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...
after: здраствуйте . я , кирилл . хотел бы чтобы вы сделали игру , 3д - экшон суть такова ...


__Задание:__ Токенизируйте title и description в train и test

In [0]:
X_train[:, 0] = [preprocess(X_train[i][0]) for i in range(X_train.shape[0])] #tokenize title 

In [0]:
X_train[:, 1] = [preprocess(X_train[i][1]) for i in range(X_train.shape[0])] #tokenize description

In [0]:
assert X_train[10][1].lower() == 'продам иж планета 3 , 76 год , ( стоит на старом учёте , документы утеряны ) на ходу , хорошее состояние , все интересующие вопросы по телефону ( с родной коляской на 3 тысячи дороже ) . торга не будет .'

### BOW (1.5 балла)

Один из традиционных подходов -- построение bag of words.

Метод состоит в следующем:

 - Составить словарь самых часто встречающихся слов в train data
 - Для каждого примера из train посчитать, сколько раз каждое слово из словаря в нём встречается


 В sklearn есть CountVectorizer, но в этом задании его использовать нельзя.

__Задание:__ Найдите k самых частых слов, отсортируйте их по убыванию частотности (k=10000)

In [13]:
dictionary = {}
for obj in X_train:
  for column in obj:
    for word in column.split():
      if word not in dictionary.keys():
        dictionary[word] = 1
      else:
        dictionary[word] += 1

dictionary = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)
most_common = dict(dictionary[:10000])
bow_vocabulary = [item[0] for item in dictionary[:10000]]
print(len(bow_vocabulary))

10000


In [0]:
assert sorted(bow_vocabulary)[::200] == ['!', '12500', '270', '700', 'by', 'gh', 'michael', 'sonata', 'ø', 'аудиоподготовка', 'большим', 'веса', 'воспроизведения', 'габариты', 'гтд', 'джинсами', 'доступность', 'загрузки', 'зимней', 'использовался', 'квартала', 'коммуникации', 'кошки', 'лакированные', 'магазин', 'металл', 'мск', 'натуральным', 'носке', 'одному', 'отвечаем', 'пассат', 'плотно', 'покраску', 'постоянные', 'примеры', 'просьба', 'размещайте', 'репетитор', 'сантехник', 'сидения', 'современного', 'стала', 'схема', 'тон', 'удлиненная', 'фасад', 'цветами', 'шея', 'эту']

In [0]:
def text_to_bow(text: str) -> np.array:
    """
    Возвращает вектор, где для каждого слова из most_common
    указано количество его употреблений
    """ 
    array = np.zeros(10000)
    tokenized_str = preprocess(text).split()
    for i in range(10000):
      array[i] = tokenized_str.count(bow_vocabulary[i])
    
    return array

In [0]:
assert np.allclose(np.where(text_to_bow("сдаётся уютный , тёплый гараж для стартапов в ml") != 0)[0],
                   np.array([   1,    4,   12,  565,  866, 1601, 2539, 4063])
)

In [0]:
def items_to_bow(items: np.array) -> np.array:
    """ Для каждого товара возвращает вектор его bow """
    # Давайте для начала попробуем строить bow только из description товара
    # assert ниже написан для bow из description


    lst = []
    for item in items:
      lst.append(text_to_bow(preprocess(item[1])))

    return np.array(lst)

In [0]:
assert np.allclose(np.where(items_to_bow([X_train[42]])[0] != 0),
                   np.array([   0, 1, 2, 5, 6, 7, 12, 27, 41, 49, 110,
                                189,  208,  221, 2032, 3052, 7179, 9568]),
)

In [0]:
X_train_bow = items_to_bow(X_train)
X_test_bow = items_to_bow(X_test)

In [0]:
from sklearn.metrics import accuracy_score

### Логистическая регрессия и SVC (1 балл)


Теперь описание каждого товара представлено, как точка в многомерном пространстве.
Очень важно запомнить эту идею: дальше мы будем рассматривать разные способы перехода от текста к точке в пространстве.

Для BOW каждое измерение в пространстве -- какое-то слово.
Мы предполагаем, что текст описывается набором каких-то популярных слов, которые в нём встречаются, а близкие по смыслу тексты будут использовать одинаковые слова.

Обучите логистическую регрессию и SVC с базовыми параметрами.


In [21]:
from sklearn.linear_model import LogisticRegression
bow_model = LogisticRegression(max_iter=100).fit(X_train_bow, y_train)
print(accuracy_score(bow_model.predict(X_test_bow), y_test))

assert accuracy_score(bow_model.predict(X_test_bow), y_test) > 0.7

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7011111111111111


In [22]:
from sklearn.svm import LinearSVC

bow_model = LinearSVC(max_iter=70).fit(X_train_bow, y_train)
print(accuracy_score(bow_model.predict(X_test_bow), y_test))

assert accuracy_score(bow_model.predict(X_test_bow), y_test) > 0.68

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.6843333333333333


### Модификация признаков (0.5 балла)

Добавьте title товара в bow с произвольным весом, как изменится качество?

In [0]:
def items_to_bow(items: np.array) -> np.array:
    """ Для каждого товара возвращает вектор его bow """

    lst = []
    for item in items:
      lst.append(text_to_bow(preprocess(item[1])) + 0.3 * text_to_bow(preprocess(item[0])))

    return np.array(lst)

In [0]:
X_train_bow = items_to_bow(X_train)
X_test_bow = items_to_bow(X_test)

In [25]:
bow_model = LogisticRegression(max_iter=100).fit(X_train_bow, y_train)
print(accuracy_score(bow_model.predict(X_test_bow), y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7348888888888889


In [26]:
bow_model = LinearSVC(max_iter=70).fit(X_train_bow, y_train)
print(accuracy_score(bow_model.predict(X_test_bow), y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.7243333333333334


Качество возрасло 3,3% и 4%, так как теперь мы обучаемся не только на данных из description, но и на данных из title, что дает нам больше информации для обобщения

Нормализуйте данные (`sklearn.preprocessing.normalize`) перед обучением. Что станет с качеством и почему?

In [27]:
from sklearn.preprocessing import normalize
X_train_bow_norm = normalize(X_train_bow)

bow_model = LogisticRegression(max_iter=100).fit(X_train_bow_norm, y_train)
print(accuracy_score(bow_model.predict(X_test_bow), y_test))

bow_model = LinearSVC(max_iter=70).fit(X_train_bow_norm, y_train)
print(accuracy_score(bow_model.predict(X_test_bow), y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.4641111111111111
0.696


Качество упало на обеих моделях, на 27% и 3%. Возможная причина падения качества заключается в том, что при нормализации скорость схождения градиента увеличилась и стало требоваться меньше шагов для обучения, но так как количество шагов зафиксировано, то произошло переобучение

### mystem (0.5) балла

Попробуйте обучиться, используя токенизатор mystem. Сравните качество.

In [28]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /bin

--2020-04-26 17:28:08--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.244, 5.45.205.242, 5.45.205.245, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.244|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskm909.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2020-04-26 17:28:08--  http://cache-mskm909.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-mskm909.cdn.yandex.net (cache-mskm909.cdn.yandex.net)... 5.45.220.19, 2a02:6b8:0:2002::1a
Connecting to cache-mskm909.cdn.yandex.net (cache-mskm909.cdn.yandex.net)|5.45.220.19|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz’

mystem-3.0-linux3.1 100%[===================>]  1

In [29]:
pip install git+https://github.com/nlpub/pymystem3

  Cloning https://github.com/nlpub/pymystem3 to /tmp/pip-req-build-9zjdrsne
  Running command git clone -q https://github.com/nlpub/pymystem3 /tmp/pip-req-build-9zjdrsne
  Created wheel for pymystem3: filename=pymystem3-0.2.0-cp36-none-any.whl size=9921 sha256=bb2ac96f1bb067b983122683d0af4207f5ef9d018874ca25febb77db3f9c1caa
  Stored in directory: /tmp/pip-ephem-wheel-cache-_xf2llvb/wheels/7d/75/c2/216a594291dee680749ce12c60d16125cfe1f363059e7163dc
Successfully built pymystem3


In [0]:
from pymystem3 import Mystem

m = Mystem()
dictionary = {}
for obj in X_train:
    for word in ''.join(m.lemmatize(obj[0] + obj[1])).split():
      if word not in dictionary.keys():
        dictionary[word] = 1
      else:
        dictionary[word] += 1

dictionary = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)
bow_vocabulary = [item[0] for item in dictionary[:10000]]

def text_to_bow(text: str) -> np.array:
    """
    Возвращает вектор, где для каждого слова из most_common
    указано количество его употреблений
    """ 
    array = np.zeros(len(bow_vocabulary))
    tokenized_str = ''.join(m.lemmatize(text)).split()
    for i in range(len(bow_vocabulary)):
      array[i] = tokenized_str.count(bow_vocabulary[i])
    
    return array

def items_to_bow(items: np.array) -> np.array:
    """ Для каждого товара возвращает вектор его bow """


    lst = []
    for item in items:
      lst.append(text_to_bow(item[1] + item[0]))

    return np.array(lst)


In [0]:
X_train_bow = items_to_bow(X_train)
X_test_bow = items_to_bow(X_test)


In [32]:
bow_model = LogisticRegression(max_iter=100).fit(X_train_bow, y_train)
print('LogisticRegression:', accuracy_score(bow_model.predict(X_test_bow), y_test))

bow_model = LinearSVC(max_iter=70).fit(X_train_bow, y_train)
print('LinearSVC:', accuracy_score(bow_model.predict(X_test_bow), y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression: 0.6771111111111111


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC: 0.655


Качество упало на 6% и 7%, по сравнению с обычным разбиением по пробелам.

### TF-IDF (1.5 балла)

Не все слова полезны одинаково, давайте попробуем [взвесить](http://tfidf.com/) их, чтобы отобрать более полезные.


> TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).
> 
> IDF(t) = log_e(Total number of documents / Number of documents with term t in it).


В sklearn есть TfidfVectorizer, но в этом задании его использовать нельзя.

In [0]:
# Давайте для простоты считать один tf-idf для title и description.
# Для каждого слова из bow_vocabulary нужно посчитать
# в тексте скольких товаров встретилось это слово

In [0]:
from math import log1p

def text_to_tfidf(array: np.array) -> np.array:
    """
    Возвращает вектор, где для каждого слова из most_common
    указан tf-idf
    """
    m = Mystem()
    words = [''.join(m.lemmatize(obj[0] + obj[1])).split() for obj in array]
    count_arr = []
    for word in bow_vocabulary:
      count = 0
      for i in range(len(array)):
        if word in words[i]:
          count += 1
      count_arr.append(count)
    count_arr = np.array(count_arr, dtype='int64')
    count_arr = np.nan_to_num(count_arr)

    result = []

    num_of_docs = np.array([array.shape[0] for _ in range(len(bow_vocabulary))])
    for obj in array:
      words = ''.join(m.lemmatize(obj[0] + obj[1])).split()
      num_appear = []
      num_of_terms = [len(words) for _ in range(len(bow_vocabulary))]
      for word in bow_vocabulary:
        num_appear.append(words.count(word))
      result.append(np.array(num_appear) / np.array(num_of_terms) * np.array([log1p(i/j) for i, j in zip(num_of_docs, count_arr)]))
    result = np.array(result)
    result = np.nan_to_num(result)

    return result

In [35]:
# Нормализуйте данные
from sklearn.preprocessing import normalize

X_train_norm = normalize(text_to_tfidf(X_train))
X_test_norm = normalize(text_to_tfidf(X_test))

[ 9319 14629 15667 ...     4     4     6]
[2825  552  360 ...    1    0    0]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in long_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in multiply


### Модели на TF-IDF признаках (1 балл)

Обучите логистическую регрессию и SVC, оцените качество (accuracy_score)

In [36]:
bow_model = LogisticRegression(max_iter=100).fit(X_train_norm, y_train)
print('LogisticRegression:', accuracy_score(bow_model.predict(X_test_norm), y_test))

bow_model = LinearSVC(max_iter=70).fit(X_train_norm, y_train)
print('LinearSVC:', accuracy_score(bow_model.predict(X_test_norm), y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression: 0.6652222222222223
LinearSVC: 0.7245555555555555


### Hashing Vectorizer (0.5 балла)

Попробуйте использовать `sklearn.feature_extraction.text.HashingVectorizer` для векторизации текстов.
Обязательно оцените качество работы алгоритмов классификации с использованием новой векторизации.

In [37]:
from sklearn.feature_extraction.text import HashingVectorizer

corpus_train = [obj[0] + obj[1] for obj in X_train]
corpus_test = [obj[0] + obj[1] for obj in X_test]

vectorizer = HashingVectorizer(n_features=2**20, analyzer='word')
X_train_test = vectorizer.fit_transform(corpus_train)
X_test_test = vectorizer.fit_transform(corpus_test)

bow_model = LogisticRegression(max_iter=100).fit(X_train_test, y_train)
print('LogisticRegression:', accuracy_score(bow_model.predict(X_test_test), y_test))

bow_model = LinearSVC(max_iter=70).fit(X_train_test, y_train)
print('LinearSVC:', accuracy_score(bow_model.predict(X_test_test), y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression: 0.6961111111111111
LinearSVC: 0.7921111111111111


Качество на LinearSVC значительно выросло (на 6% по сравнению с лучшим результатом), на LogReg упало на 3%

### Word Vectors (1 балл)

Давайте попробуем другой подход -- кажому слову сопоставим какой-то эмбеддинг (вектор).

Вектора будут небольшой размерности. Таким образом мы снизим количество параметров в модели.

Вектора мы возьмём уже готовые (обученные на текстах их интернета), так что наша модель будет знать некоторую дополнительную информацию о внешнем мире.

In [38]:
!wget https://www.dropbox.com/s/0x7oxso6x93efzj/ru.tar.gz

--2020-04-26 18:09:10--  https://www.dropbox.com/s/0x7oxso6x93efzj/ru.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/0x7oxso6x93efzj/ru.tar.gz [following]
--2020-04-26 18:09:10--  https://www.dropbox.com/s/raw/0x7oxso6x93efzj/ru.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc5c546df22557a9302470bd8af.dl.dropboxusercontent.com/cd/0/inline/A2me129Rph9v5Ay3z2GQMjy9fCfT6SqX63eANWHnUtLg5yx_ifSjDqdHnVmQ93R9jCCU2CrtAvi_MFclDRfp8Tr3GNzU2k8U8xz3Ou1MjRU9zw/file# [following]
--2020-04-26 18:09:10--  https://ucc5c546df22557a9302470bd8af.dl.dropboxusercontent.com/cd/0/inline/A2me129Rph9v5Ay3z2GQMjy9fCfT6SqX63eANWHnUtLg5yx_ifSjDqdHnVmQ93R9jCCU2CrtAvi_MFclDRfp8Tr3GNzU2k8U8xz3Ou1MjRU9zw/file
Resolving ucc5c546df22557

In [39]:
!tar -xzvf ru.tar.gz

ru.bin
ru.vec


In [0]:
import gensim
from gensim.models.wrappers import FastText


model = FastText.load_fasttext_format('ru.bin')

In [0]:
# Эмбеддинг предложения -- сумма эмбеддингов токенов


def sentence_embedding(sentence: str) -> np.array:
    """
    Складывает вектора токенов строки sentence
    """

    embedding_dim = model['модель'].shape[0]
    features = np.zeros([embedding_dim], dtype='float32')
    
    for word in sentence.split():
        if word in model:
            features += model[word]
    
    return features

In [0]:
assert np.allclose(sentence_embedding('сдаётся уютный , тёплый гараж для стартапов в ml')[::50],
                   np.array([ 0.08189847,  0.07249198, -0.15601222,  0.03782297,  0.09215296, -0.23092946]))

In [82]:
# Обучите логистическую регрессию и SVM
# Оцените качество (accuracy_score)

m = Mystem()

X_train = np.array([sentence_embedding(''.join(m.lemmatize(obj[0] + obj[1]))) for obj in X_train])
X_test = np.array([sentence_embedding(''.join(m.lemmatize(obj[0] + obj[1]))) for obj in X_test])

bow_model = LogisticRegression(max_iter=100).fit(X_train, y_train)
print('LogisticRegression:', accuracy_score(bow_model.predict(X_test), y_test))

bow_model = LinearSVC(max_iter=70).fit(X_train, y_train)
print('LinearSVC:', accuracy_score(bow_model.predict(X_test), y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression: 0.5583333333333333
LinearSVC: 0.5445555555555556


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


### Что дальше?

Решение каждого пункта 1 балл:

1. N-Gram модели текстовой классификации

2. Обучиться на полных данных (контест на kaggle)

3. Поработать с другими эмбеддингами (word2vec, GloVe).

4. Использовать Vowpal Wabbit вместо sklearn.

5. Другие способы токенизации (pymorphy2, spaCy)


Снабжайте код пояснениями и графиками.
Обязательно необходимо написать вывод по каждому пункту, который вы реализуете.